In [12]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import Hour, Minute
from pandas.tseries.offsets import Day, MonthEnd
from pandas.tseries.offsets import Hour
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from keras.preprocessing.sequence import TimeseriesGenerator
from time import time
import xgboost as xgb

np.random.seed(42)


In [13]:

global busses
busses =        ['310_PV_1', '310_PV_2', '320_PV_1', '314_PV_1', '314_PV_2', '313_PV_1', '314_PV_3', '314_PV_4', '313_PV_2', '324_PV_1',    
                 '312_PV_1', '324_PV_2', '324_PV_3', '113_PV_1', '319_PV_1', '215_PV_1', '102_PV_1', '101_PV_1', '102_PV_2',
                 '104_PV_1', '101_PV_2', '101_PV_3', '101_PV_4', '103_PV_1', '119_PV_1']



In [14]:
df = pd.read_csv('PV_deseason_15.csv', parse_dates=[0], index_col=0)

In [15]:
def lags(df, n):
    
    df_new = df
    
    for i in range(n):
        name = "lag "+ str(i+1)
        df_new[name] = df['310_PV_1'].shift(i+1)
    
    return df_new
    

In [16]:
def TrainTest(df):
    
    num_rows = df.shape[0]
    
    num_training = (num_rows * 8)//10
   
    #train = df.iloc[0:num_training]
    #test = df.iloc[num_training:num_rows]
    
    
    return num_training 

In [17]:
def TestModel(X,test, mod):
    
    MSEs = []
    
    for i in range(len(X)):
        
        print(X.iloc[i].to_frame().transpose())
        pred = mod.predict(X.iloc[i].to_frame().transpose())
        #pred = scaler.inverse_transform(pred)
        
        print(pred)
        print(test.iloc[i])
        
        MSEs.append(mse([test.iloc[i]],pred))
    
        
        
    return MSEs

In [18]:



def model1Train(X_train,Y_train,X_test,Y_test):
    mod = xgb.XGBRegressor()
    mod.fit(X_train, Y_train, eval_set = [(X_train, Y_train),(X_test, Y_test)])#keep low for now, edit later eval_set = [(X_train, y_train),(X_test, y_test)]
    
    return mod

    

In [26]:
def model2Train(X_train,Y_train):
    mod = xgb.XGBRegressor()
    start = time()

    mod.fit(X_train, Y_train)#keep low for now, edit later eval_set = [(X_train, y_train),(X_test, y_test)]
    print('time')
    print(time()-start)
    return mod

In [20]:
def runModels(df, last):
    scaler = MinMaxScaler()
     
    bound = TrainTest(df)
    
    
    last_index =pd.DatetimeIndex(data = [idx.replace(year = 2019) for idx in last.index])
    df_index = df.index
    new_df_idx = last_index.append(df_index)
    
    array_df = df['310_PV_1'].to_numpy()
    
    array_df = np.append(last['310_PV_1'].to_numpy(), array_df)
    
    
    
    new_df = pd.DataFrame(array_df, columns = ['310_PV_1'])
    new_df['Dates'] = new_df_idx
    new_df = new_df.set_index('Dates')
    

    
    
    lag = lags(new_df, 107)
    
    lag = lag[lag.columns[::-1]]
    

    
    
    lagged_train = lag.iloc[107:bound+107]
    lagged_test = lag.iloc[bound+107:]

    X_train = lagged_train.drop('310_PV_1', axis = 1)
    Y_train = lagged_train['310_PV_1']
    
    X_test = lagged_test.drop('310_PV_1', axis = 1)
    Y_test = lagged_test['310_PV_1']
    
    print(Y_train[1])
    print(Y_test[1])
  
    
    #mod = model1Train(X_train, Y_train,X_test, Y_test)
    mod = model2Train(X_train, Y_train)
    
    Tests = TestModel(X_test, Y_test, mod)
    
    print(Tests)
    
    return Tests

    

In [22]:

df1 = df[df.index.month.isin([1,2,3])]
df1 = df1.iloc[:-1 , :]
print(df1)
df2 = df[df.index.month.isin([4,5,6])]
df3 = df[df.index.month.isin([7,8,9])]
df4 = df[df.index.month.isin([10,11,12])]

                     320_PV_1  314_PV_1  314_PV_2  313_PV_1  314_PV_3  \
Date                                                                    
2020-01-01 00:15:00  3.966453  3.980737  4.282982  7.276192  6.895283   
2020-01-01 00:30:00  3.966107  3.980367  4.282545  7.275664  6.894732   
2020-01-01 00:45:00  3.965761  3.979998  4.282108  7.275137  6.894181   
2020-01-01 01:00:00  3.965415  3.979628  4.281671  7.274609  6.893630   
2020-01-01 01:15:00  3.965069  3.979259  4.281233  7.274081  6.893079   
...                       ...       ...       ...       ...       ...   
2020-03-31 22:45:00  0.880274  0.763739  0.579910  2.526283  2.095675   
2020-03-31 23:00:00  0.879928  0.763376  0.579482  2.525721  2.095125   
2020-03-31 23:15:00  0.879581  0.763014  0.579053  2.525159  2.094575   
2020-03-31 23:30:00  0.879234  0.762651  0.578625  2.524598  2.094024   
2020-03-31 23:45:00  0.878887  0.762289  0.578196  2.524036  2.093474   

                     314_PV_4  313_PV_2  310_PV_1 

In [27]:
mse1 = runModels(df1,df4.iloc[-107:])

C:\Users\bvits\AppData\Local\Temp\ipykernel_22424\239345827.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[name] = df['310_PV_1'].shift(i+1)
C:\Users\bvits\AppData\Local\Temp\ipykernel_22424\239345827.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[name] = df['310_PV_1'].shift(i+1)
C:\Users\bvits\AppData\Local\Temp\ipykernel_22424\239345827.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

4.125291466810914
1.6454898259911186
time
2.6642746925354004
                     lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-13 19:15:00 -2.61448 -1.724143  4.739966 -6.519696 -6.418702   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-13 19:15:00 -4.580824 -2.443491 -0.402061  1.296474  1.654844  ...   

                        lag 10      lag 9     lag 8     lag 7     lag 6  \
2020-03-13 19:15:00 -15.331883 -12.029519 -5.359668 -6.317959 -4.866415   

                        lag 5     lag 4     lag 3     lag 2     lag 1  
2020-03-13 19:15:00 -2.831537 -0.714418  0.902427  1.503425  1.641035  

[1 rows x 107 columns]
[-0.9434131]
1.6458579469421952
                      lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-13 19:30:00 -1.724143  4.739966 -6.519696 -6.418702 -4.580824   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-13 19:30:00 -2.443491 -0.402061  1.296474  1.654844  

[1.4338756]
1.6414390537380277
                      lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-13 22:30:00  1.677944  1.677571  1.677198  1.676826  1.676453   

                     lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-13 22:30:00  1.67608  1.675707  1.675334  1.674961  1.674588  ...   

                       lag 10     lag 9     lag 8     lag 7     lag 6  \
2020-03-13 22:30:00  1.644754  1.644385  1.644017  1.643649  1.643281   

                        lag 5     lag 4     lag 3     lag 2     lag 1  
2020-03-13 22:30:00  1.642912  1.642544  1.642176  1.641807  1.641439  

[1 rows x 107 columns]
[1.4338756]
1.6410707268167697
                      lag 107   lag 106   lag 105   lag 104  lag 103  \
2020-03-13 22:45:00  1.677571  1.677198  1.676826  1.676453  1.67608   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-13 22:45:00  1.675707  1.675334  1.674961  1.674588  1.674215  ...   

                   

[1.5940652]
1.6366430223751929
                      lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-14 01:45:00  1.673096  1.672723  1.672351  1.671978  1.671605   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-14 01:45:00  1.673778  1.673411  1.673044  1.672678  1.672311  ...   

                       lag 10     lag 9     lag 8     lag 7     lag 6  \
2020-03-14 01:45:00  1.639964  1.639595  1.639226  1.638857  1.638488   

                        lag 5    lag 4     lag 3     lag 2     lag 1  
2020-03-14 01:45:00  1.638119  1.63775  1.637381  1.637012  1.636643  

[1 rows x 107 columns]
[0.9495398]
1.6362739229370131
                      lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-14 02:00:00  1.672723  1.672351  1.671978  1.671605  1.673778   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-14 02:00:00  1.673411  1.673044  1.672678  1.672311  1.671944  ...   

                 

[-16.476019]
-22.87966790428232
                      lag 107  lag 106   lag 105   lag 104   lag 103  \
2020-03-14 08:00:00  1.432834  0.33438 -0.641124 -2.554333 -5.331749   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-14 08:00:00 -7.395231 -9.624286 -2.197877  4.395471  7.067782  ...   

                       lag 10     lag 9    lag 8     lag 7     lag 6  \
2020-03-14 08:00:00  0.233207 -0.813062 -2.96425 -5.632651 -7.904877   

                         lag 5    lag 4      lag 3      lag 2      lag 1  
2020-03-14 08:00:00 -10.114808 -8.53758 -16.650517 -19.742688 -22.879668  

[1 rows x 107 columns]
[-16.119278]
-25.78905247713225
                     lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-14 08:15:00  0.33438 -0.641124 -2.554333 -5.331749 -7.395231   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-14 08:15:00 -9.624286 -2.197877  4.395471  7.067782  8.883261  ...   

             

...

[3.2213004]
1.5279182315137805
                      lag 107  lag 106   lag 105   lag 104   lag 103  \
2020-03-17 05:15:00  1.566766   1.5664  1.566035  1.565669  1.565304   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-17 05:15:00  1.564939  1.564573  1.564208  1.563842  1.563477  ...   

                       lag 10     lag 9     lag 8     lag 7     lag 6  \
2020-03-17 05:15:00  1.531229  1.530861  1.530493  1.530126  1.529758   

                       lag 5     lag 4     lag 3     lag 2     lag 1  
2020-03-17 05:15:00  1.52939  1.529022  1.528654  1.528286  1.527918  

[1 rows x 107 columns]
[2.7256377]
1.2556924188929257
                     lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-17 05:30:00   1.5664  1.566035  1.565669  1.565304  1.564939   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-17 05:30:00  1.564573  1.564208  1.563842  1.563477  1.563111  ...   

                     

[4.4358773]
2.8654834178865904
                      lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-17 08:30:00 -0.744816 -2.658024 -5.435438 -7.498919 -6.061306   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-17 08:30:00  2.931771  5.291787  6.864099  8.012914  8.211728  ...   

                       lag 10     lag 9     lag 8     lag 7     lag 6  \
2020-03-17 08:30:00 -3.067816 -5.736217 -8.008443 -6.651707  2.325521   

                        lag 5     lag 4   lag 3     lag 2     lag 1  
2020-03-17 08:30:00  4.579251  5.320413  4.3501  3.840715  2.865483  

[1 rows x 107 columns]
[3.5425742]
2.86320267017328
                      lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-17 08:45:00 -2.658024 -5.435438 -7.498919 -6.061306  2.931771   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-17 08:45:00  5.291787  6.864099  8.012914  8.211728  8.997975  ...   

                     

[9.388579]
10.464699073441018
                       lag 107    lag 106    lag 105    lag 104   lag 103  \
2020-03-17 12:00:00  12.687222  12.563359  12.163813  12.139403  11.98931   

                       lag 102    lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-17 12:00:00  11.284846  10.435027  9.738486  6.712164  6.387208  ...   

                       lag 10     lag 9     lag 8     lag 7     lag 6  \
2020-03-17 12:00:00  6.799981  8.385132  8.762086  8.881117  8.656705   

                       lag 5     lag 4      lag 3      lag 2      lag 1  
2020-03-17 12:00:00  9.22273  9.333564  10.589206  10.454138  10.464699  

[1 rows x 107 columns]
[8.7518635]
11.689467303563752
                       lag 107    lag 106    lag 105   lag 104    lag 103  \
2020-03-17 12:15:00  12.563359  12.163813  12.139403  11.98931  11.284846   

                       lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-17 12:15:00  10.435027  9.738486  6.712164  6.387208  6.409519

[5.6797657]
8.64706580610899
                      lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-17 15:15:00  7.099951  8.231279  9.329546  5.683825 -5.665448   

                     lag 102  lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-17 15:15:00  7.87435  3.80677  7.311595 -6.144509  5.318513  ...   

                        lag 10     lag 9     lag 8     lag 7     lag 6  \
2020-03-17 15:15:00  10.158307  9.975698  10.62287  11.53425  11.87104   

                         lag 5      lag 4      lag 3     lag 2     lag 1  
2020-03-17 15:15:00  12.123404  10.914292  10.037421  9.048801  8.647066  

[1 rows x 107 columns]


KeyboardInterrupt: 

In [ ]:
mse2 = runModels(df2,df1.iloc[-107:])

In [ ]:
mse3 = runModels(df3,df2.iloc[-107:])

In [ ]:
mse4 = runModels(df4,df3.iloc[-107:])

In [ ]:
print('season 1')

print('model 1:')
print(mse1)



In [ ]:
print('season 1')

print('model 1:')
print(mse2)


In [ ]:
print('season 1')

print('model 1:')
print(mse3)



In [ ]:
print('season 1')

print('model 1:')
print(mse4)



In [ ]:
d = {'Season 1':mse1,'Season 2':mse2,'Season 3':mse3,'Season 4':mse4}
df_errors_GBDT = pd.DataFrame({k:pd.Series(v) for k,v in d.items()})

#df_errors_GBDT.to_csv('GBDT_errors.csv')
